<a href="https://colab.research.google.com/github/zahere-dev/openai-agents-sdk-tutorial/blob/main/openai_agents_sdk_tutorial1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Imports**

In [ ]:
! pip install openai-agents nest_asyncio colorama


In [37]:
import os
from google.colab import userdata
import asyncio
from agents import Agent, Runner, function_tool, set_default_openai_key, set_tracing_disabled
from colorama import Fore, Style


set_tracing_disabled(True)
set_default_openai_key(userdata.get("OPENAI_API_KEY"))


## **Order Agent**


In [18]:
import nest_asyncio # required for notebooks
nest_asyncio.apply()

@function_tool
def check_order_status(order_id: str):
    """Check the status of an order with the given order ID."""
    order_statuses = {
        "12345": "Your order 12345 is being prepared and will be delivered in 20 minutes.",
        "67890": "Your order 67890 has been dispatched and will arrive in 10 minutes.",
        "11121": "Your order 11121 is still being processed. Please wait a little longer."
    }
    return order_statuses.get(order_id, "Order ID not found. Please check and try again.")

order_agent = Agent(
    name="OrderAgent",
    instructions="Help customers with their order status. If they provide an order ID, fetch the status.",
    tools=[check_order_status]
)



## **FAQ Agent**


In [19]:
@function_tool
def answer_faq(question: str):
    """Ensure the input is either hours, menu, location, contact, reservation, delivery or allergies """

    faq_responses = {
        "hours": "We are open from 10 AM to 11 PM every day.",
        "menu": "You can find our menu at restaurant.com/menu.",
        "location": "We are located at 123 Main Street, Cityville.",
        "contact": "You can reach us at 555-1234 or email support@restaurant.com.",
        "reservation": "We accept reservations online at restaurant.com/reservations or by calling 555-1234.",
        "delivery": "We offer delivery through our website and on major food delivery platforms like Uber Eats and DoorDash.",
        "allergies": "We accommodate allergies! Please let us know your dietary restrictions when placing an order."
    }
    return faq_responses.get(question.lower(), "I'm not sure, but you can call our helpline at 555-1234.")

faq_agent = Agent(
    name="FAQAgent",
    instructions="Answer common customer questions about hours, menu, and location.\
     Augment the answer based on the tone and details requested in the query \
     Pick up the relevant keyword from the user's query and pass that as input. \
     Example: If user is asking about time then the input keyword is hours.",
    tools=[answer_faq]
)

## **Complaint Handler Agent**


In [20]:
@function_tool
def handle_complaint(complaint: str):
    """Handle customer complaints and ensure respectful communication."""
    return "Thank you for your feedback. We take complaints seriously and will address your concern as soon as possible."

complaint_agent = Agent(
    name="ComplaintAgent",
    instructions="Handle customer complaints and ensure respectful communication.",
    tools=[handle_complaint]
)



## **Reservation Agent**

In [28]:
@function_tool
def handle_reservation(request: str):
    """Ensure the input is either make, modify, cancel or availability """
    reservation_responses = {
        "make": "Your reservation request has been received. Please check your email for confirmation.",
        "modify": "Your reservation modification request has been received. Please check your email for updates.",
        "cancel": "Your reservation has been canceled. We hope to see you another time!",
        "availability": "We have availability for dinner slots from 6 PM to 9 PM. Please book online or call us."
    }
    return reservation_responses.get(request.lower(), "I'm not sure about that request. Please call us at 555-1234 for assistance.")

reservation_agent = Agent(
    name="ReservationAgent",
     instructions="Assist customers with making, modifying, or canceling reservations.\
     Pick up the relevant keyword from the user's query and pass that as input. \
     Example: If user is asking making a reservation then input keyword is make.",
    tools=[handle_reservation]
)

In [31]:
classifier_agent = Agent(
    name="User Interface Agent",
    model="gpt-4o-mini",
    instructions="Handoff to appropriate agent based on user query",
    handoffs=[order_agent,faq_agent,complaint_agent,reservation_agent]
)


# **Chat**

In [ ]:
async def chat():
    print("Welcome to the Restaurant Customer Support chat! Type 'exit' to end the chat.")
    response = ""
    while True:
        user_input = input(Fore.GREEN + "You: " + Style.RESET_ALL)
        if user_input.lower() == "exit":
            print(Fore.RED + "Goodbye!" + Style.RESET_ALL)
            break

        if response:
          input_with_context = response.to_input_list() + [
          {"role": "user", "content": user_input}
          ]
        else:
          input_with_context = [{"role": "user", "content": user_input}]
        response = await Runner.run(classifier_agent, input=input_with_context)
        print(Fore.BLUE + f"Support Agent: {response.final_output}" + Style.RESET_ALL)



if __name__ == "__main__":
    asyncio.run(chat())


Welcome to the Restaurant Customer Support chat! Type 'exit' to end the chat.
You: What is the status of order 12345
Support Agent: Your order 12345 is being prepared and will be delivered in 20 minutes.
You: When are you guys open?
Support Agent: We are open from 10 AM to 11 PM every day.
You: I want to book a table for 2 at 2PM today
Support Agent: Your reservation request has been received. Please check your email for confirmation.
You: What is the status of order klsdjfklsdjf
Support Agent: The order ID "klsdjfklsdjf" was not found. Please check it and try again.
You: 11121
Support Agent: Your order 11121 is still being processed. Please wait a little longer.
You: Are you open at 9AM?
Support Agent: No, we open at 10 AM.
You: Are you guys present in London as well?
Support Agent: We're currently located at 123 Main Street, Cityville. London isn't one of our locations yet!
You: Do you remember the time of my reservation?
Support Agent: Yes, your reservation is for 2 PM today.
You: C